Calcular los 'valores gatillo' para las variables climáticas seleccionadas en un año específico.

In [ ]:
import pandas as pd
import numpy as np
import os

# Definir la ruta de la carpeta 'data'
data_dir = os.path.join('data')


# Cargar los datos climáticos desde un archivo CSV.
# Asegúrate de que el archivo 'dataset_modificado.csv' esté en la misma carpeta que este código.
ruta_csv = "dataset_modificado.csv"
df_variables = pd.read_csv(ruta_csv)

def calcular_valores_gatillo(df, anio, variables, percentil=0.95):
    """
    Calcula los 'valores gatillo' para las variables climáticas seleccionadas en un año específico.
    
    Parámetros:
    - df: Conjunto de datos con información climática y de provincias.
    - anio: Año para el cual se quieren calcular los valores gatillo.
    - variables: Lista de variables climáticas para calcular los valores gatillo (por ejemplo, temperatura máxima).
    - percentil: Percentil que se usará para calcular el valor gatillo (por defecto, el 95%).
    
    Retorna:
    - Un DataFrame con los valores gatillo por provincia y año.
    """
    # Filtrar los datos para el año seleccionado.
    df_anio = df[df['año'] == anio]
    
    # Crear un diccionario para almacenar los valores gatillo de cada provincia.
    valores_gatillo = {}
    
    # Obtener la lista de provincias disponibles en los datos.
    provincias = df_anio['provincia'].unique()
    
    for provincia in provincias:
        # Filtrar los datos para la provincia actual.
        df_provincia = df_anio[df_anio['provincia'] == provincia]
        
        # Diccionario para almacenar los valores gatillo de la provincia actual.
        valores_provincia = {}
        
        # Calcular el valor gatillo para cada variable climática seleccionada.
        for var in variables:
            # El valor gatillo es el valor por encima del cual el 5% de los datos son mayores.
            valor_gatillo = df_provincia[var].quantile(percentil)
            valores_provincia[f'gatillo_{var}'] = valor_gatillo
        
        # Guardar los valores gatillo de la provincia en el diccionario principal.
        valores_gatillo[provincia] = valores_provincia
    
    # Convertir el diccionario a un DataFrame para facilitar su uso y análisis.
    df_valores_gatillo = pd.DataFrame(valores_gatillo).T
    df_valores_gatillo.index.name = 'provincia'
    df_valores_gatillo.reset_index(inplace=True)
    df_valores_gatillo['año'] = anio
    return df_valores_gatillo

# Seleccionar las variables climáticas para las cuales queremos calcular los valores gatillo.
# Puedes añadir más variables a esta lista si lo deseas.
variables_interes = ['temp_max_max', 'humedad_max']

# Obtener los dos últimos años disponibles en los datos para hacer el análisis.
anios_disponibles = sorted(df_variables['año'].unique())[-2:]

# Lista para almacenar los valores gatillo de los dos últimos años.
valores_gatillo_acumulados = []

for anio in anios_disponibles:
    # Calcular los valores gatillo para cada año y añadirlos a la lista.
    df_gatillos_anuales = calcular_valores_gatillo(df_variables, anio, variables_interes)
    valores_gatillo_acumulados.append(df_gatillos_anuales)

# Combinar los valores gatillo de los dos años en un solo DataFrame.
df_gatillos_acumulados = pd.concat(valores_gatillo_acumulados, ignore_index=True)

# Calcular el promedio de los valores gatillo de los dos años para cada provincia.
df_gatillos_promedio = df_gatillos_acumulados.groupby('provincia').mean().reset_index()

# Mostrar los resultados en la pantalla para que el usuario pueda verlos.
print("Valores gatillo promedio de los últimos 2 años por provincia:")
print(df_gatillos_promedio)

# Guardar los valores gatillo promedio en un archivo CSV para su uso posterior.
ruta_salida_promedio = "gatillos_provincia_promedio_ultimos_2_anos.csv"
df_gatillos_promedio.to_csv(ruta_salida_promedio, index=False)

print("Los valores gatillo promedio han sido calculados y guardados en:", ruta_salida_promedio)